In [1]:
import pickle
from pacti.terms.polyhedra import *
from pacti.utils import read_contracts_from_file
from generators import *

def show_bounds(c: PolyhedralContract, var:str):
    b = c.get_variable_bounds(var=var)
    print(f"{var} bounds={b}")
    
f20 = open('data/results20.data', 'rb')
results20 = pickle.load(f20)
f20.close()

ranges: list[tuple2float] =results20[0][0]
# l,h=zip(*ranges)
# print(f"l={l}")
# print(f"h={h}")
# m=[(l[i] + h[i])/2 for i in range(0, len(l))]
# d=[(h[i] - l[i])/(2*m[i]) for i in range(0, len(l))]
# print(f"m={m}\nd={d}")

s1 = power_consumer(s=1, task="dsn", consumption=ranges[0])
print(f"\n=== s1\n{s1}")

s2 = CHRG_power(s=2, generation=ranges[1])
print(f"\n=== s2\n{s2}")

steps2 = scenario_sequence(c1=s1, c2=s2, variables=power_variables, c1index=1, file_name="json/s1-s2-step2.json")
print(f"\n=== steps2\n{steps2}")

pwr1 = generate_power_scenario(
        s=1,
        dsn_cons=ranges[0],
        chrg_gen=ranges[1],
        sbo_cons=ranges[2],
        tcmh_cons=ranges[3],
        tcmdv_cons=ranges[4],
        rename_outputs=False,
    )
print(f"\n=== pwr1\n{pwr1}")

pwr2 = generate_power_scenario(
        s=6,
        dsn_cons=ranges[0],
        chrg_gen=ranges[1],
        sbo_cons=ranges[2],
        tcmh_cons=ranges[3],
        tcmdv_cons=ranges[4],
        rename_outputs=False,
    )
print(f"\n=== pwr2\n{pwr2}")

pwr12 = scenario_sequence(c1=pwr1, c2=pwr2, variables=power_variables, c1index=5, file_name="json/pwr1-pwr2-pwr12.json")
print(f"\n=== pwr12\n{pwr12}")

pwr3 = generate_power_scenario(
        s=11,
        dsn_cons=ranges[0],
        chrg_gen=ranges[1],
        sbo_cons=ranges[2],
        tcmh_cons=ranges[3],
        tcmdv_cons=ranges[4],
        rename_outputs=False,
    )
print(f"\n=== pwr3\n{pwr3}")

pwr123 = scenario_sequence(c1=pwr12, c2=pwr3, variables=power_variables, c1index=10, file_name="json/pwr12-pwr3-pwr123.json")
print(f"\n=== pwr123\n{pwr123}")


=== s1
InVars: [soc1_entry, duration_dsn1]
OutVars:[soc1_exit]
A: [
  -duration_dsn1 <= 0.0
  soc1_entry <= 100.0
  -soc1_entry <= 0.0
  5.216612506625804 duration_dsn1 - soc1_entry <= 0.0
]
G: [
  -5.216612506625804 duration_dsn1 + soc1_entry - soc1_exit <= 0.0
  1.513962128608989 duration_dsn1 - soc1_entry + soc1_exit <= 0.0
]

=== s2
InVars: [soc2_entry, duration_charging2]
OutVars:[soc2_exit]
A: [
  -duration_charging2 <= 0.0
  soc2_entry <= 100.0
  -soc2_entry <= 0.0
  5.604821979031521 duration_charging2 + soc2_entry <= 100.0
]
G: [
  -5.604821979031521 duration_charging2 - soc2_entry + soc2_exit <= 0.0
  2.1727035334442464 duration_charging2 + soc2_entry - soc2_exit <= 0.0
]

=== steps2
InVars: [soc1_entry, duration_dsn1, duration_charging2]
OutVars:[soc2_exit, output_soc1]
A: [
  -duration_charging2 <= 0.0
  5.604821979031521 duration_charging2 - 1.51396212860899 duration_dsn1 + soc1_entry <= 100.0
  -duration_dsn1 <= 0.0
  soc1_entry <= 100.0
  5.216612506625804 duration_dsn1

In [2]:
contracts = ["s1", "s2", "steps2", "pwr1", "pwr2", "pwr12", "pwr3", "pwr123"]

for contract in contracts:
    print(f"== Bounds for contract {contract}")
    for variable in globals()[contract].outputvars:
        print(f"  {variable}: {pwr123.get_variable_bounds(str(variable))}")

== Bounds for contract s1
  soc1_exit: (None, None)
== Bounds for contract s2
  soc2_exit: (None, None)
== Bounds for contract steps2
  soc2_exit: (None, None)
  output_soc1: (0.0, 100.0)
== Bounds for contract pwr1
  output_soc1: (0.0, 100.0)
  output_soc2: (0.0, 100.0)
  output_soc3: (None, 100.0)
  soc5_exit: (None, None)
  output_soc4: (None, 100.0)
== Bounds for contract pwr2
  output_soc6: (None, 100.0)
  output_soc7: (None, None)
  output_soc8: (None, None)
  soc10_exit: (None, None)
  output_soc9: (None, None)
== Bounds for contract pwr12
  output_soc1: (0.0, 100.0)
  output_soc2: (0.0, 100.0)
  output_soc3: (None, 100.0)
  output_soc4: (None, 100.0)
  output_soc6: (None, 100.0)
  output_soc7: (None, None)
  output_soc8: (None, None)
  soc10_exit: (None, None)
  output_soc9: (None, None)
  output_soc5: (None, 100.0)
== Bounds for contract pwr3
  output_soc11: (None, None)
  output_soc12: (None, None)
  output_soc13: (None, None)
  soc15_exit: (None, None)
  output_soc14: (None,

In [3]:
terms = pwr123.a.get_terms_with_vars([Var("duration_charging12"), Var("duration_dsn11")]).terms
for term in terms:
    print(term)

-1.0*duration_charging12 <= 0.0
5.604821979031521*duration_charging12 + -1.51396212860899*duration_dsn11 <= 0.0
